# 문제 정의

- 보스턴 주택 가격 예측
- 특성공학과 회귀모델(랜덤 포레스트와 부스팅)

## 라이브러리 로딩

In [1]:
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mglearn
import seaborn as sns

# 음수표현 라이브러리
plt.rcParams['axes.unicode_minus'] = False

# 경고무시
import warnings
warnings.filterwarnings("ignore")

# 스타일
sns.set_style('darkgrid') 

# 매직명령어 : 시각화 결과가 노트북에 포함되도록
%matplotlib inline

In [2]:
pd.options.display.max_columns = 200
pd.set_option('display.max_rows', 200)

## 데이터셋 로딩

In [3]:
# 소수점 표현 제한
pd.set_option('display.float_format', lambda x : '{:.3f}'.format(x))

In [4]:
path = "C:/k_digital/Machine Learning with Python/source/house_price/"
train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/k_digital/Machine Learning with Python/source/house_price/train.csv'

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.head()

In [ ]:
train_ID = train_df['Id']
test_ID = test_df['Id']

train_df.drop('Id', axis=1, inplace=True)
test_df.drop('Id', axis=1, inplace=True)

## 데이터 전처리

### 이상치 분석

- GrLivArea: Above grade (ground) living area square feet
- SalePrice - the property's sale price in dollars. This is the target variable that you're trying to predict.

In [ ]:
plt.figure(figsize=(12, 12))
plt.scatter(x = train_df['GrLivArea'], y= train_df['SalePrice'])
plt.xlabel('GrLivArea')
plt.ylabel('SalePrice')
plt.show()

In [ ]:
train_df = train_df.drop(train_df[(train_df['GrLivArea']>4000) & (train_df['SalePrice']<300000)].index) 

In [ ]:
plt.figure(figsize=(12, 12))
plt.scatter(x = train_df['GrLivArea'], y= train_df['SalePrice'])
plt.xlabel('GrLivArea')
plt.ylabel('SalePrice')
plt.show()

### Target : SalePrice분석

- Q-Q Plot : 두 데이터 집단 간의 분포 체크
- Q-Q wiki
- Q-Q plot 한글 블로그 자료 : sw4r님

In [ ]:
from scipy import stats
from scipy.stats import norm, skew

sns.distplot(train_df['SalePrice'], fit=norm)

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train_df['SalePrice'])
print(mu, sigma)

# 분포를 그래프에 그려봅시다
plt.legend(['Normal dist. ($\mu$={:.2f} and $\sigma$={:.2f})'.format(mu,sigma)], loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

# QQ-plot을 그려봅시다.
fig = plt.figure()
res = stats.probplot(train_df['SalePrice'], plot=plt)
plt.show()

- 분포가 오른쪽으로 치우친 경향을 보인다.
- 선형 모델을 사용할 경우 분포가 균형잡힌 상태에 더 용이하다

In [ ]:
# 데이터의 정규화를 위한 numpy의 log1p 함수를 사용
train_df['SalePrice'] = np.log1p(train_df['SalePrice'])

# 위에서와 같은 코드로 똑같이 분포를 확인해봅니다.
sns.distplot(train_df['SalePrice'], fit=norm)
(mu, sigma) = norm.fit(train_df['SalePrice'])
print(mu, sigma)
plt.legend(['Normal dist. ($\mu$={:.2f} and $\sigma$={:.2f})'.format(mu,sigma)], loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')
fig = plt.figure()
res = stats.probplot(train_df['SalePrice'], plot=plt)
plt.show()

- 정규분포에 매우 근접하게 값들이 바뀐것을 확인할 수 있다.

### Feature Engineering

In [ ]:
test_df.columns

In [ ]:
y_train = train_df.SalePrice.values

train_df.drop(['SalePrice'], axis=1, inplace=True)

- 결측치 처리

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.info()

In [ ]:
corr = train_df.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(corr, vmax=0.9, square=True)
plt.show()

- 분석을 위해 train, test 데이터셋 합치기

In [ ]:
all_data = pd.concat((train_df, test_df)).reset_index(drop=True)
#all_data.drop(['SalePrice'], axis=1, inplace=True)
all_data.shape

### 결측치 처리

- 수치형 데이터의 경우 0, 범주형 데이터의 경우 'None'로 대체

In [ ]:
rate = (all_data.isnull().sum() / len(all_data)) * 100
rate = rate.drop(rate[rate == 0].index).sort_values(ascending=False)

missing_data = pd.DataFrame({'Missing Rate':rate})
missing_data

- PoolQC : NA값은 'No Pool'을 의미	

In [ ]:
all_data['PoolQC'] = all_data['PoolQC'].fillna('None')

- MiscFeature, Alley, Fence, FireplaceQu : 없음을 의미 

In [ ]:
all_data['MiscFeature'] = all_data['MiscFeature'].fillna('None')
all_data['Alley'] = all_data['Alley'].fillna('None')
all_data['Fence'] = all_data['Fence'].fillna('None')
all_data['FireplaceQu'] = all_data['FireplaceQu'].fillna('None')

- LotFrontage : Linear feet of street connected to property
- 거리와 집의 거리를 나타냄
- 이웃들의 거리와 유사하다고 판단하여 이웃들의 중앙값으로 대체

In [ ]:
all_data['LotFrontage'] = all_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x : x.fillna(x.median()))

- GarageType, GarageFinish, GarageQual, GarageCond 모두 Na => None

In [ ]:
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    all_data[col] = all_data[col].fillna('None')

In [ ]:
for col in ['GarageYrBlt', 'GarageCars', 'GarageArea']:
    all_data[col] = all_data[col].fillna(0)

In [ ]:
col_list = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
for col in col_list:
    all_data[col] = all_data[col].fillna('None')

In [ ]:
for col in ['BsmtFinSF1','BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']:
    all_data[col] = all_data[col].fillna(0)

In [ ]:
all_data['MasVnrType'] = all_data['MasVnrType'].fillna('None')
all_data['MasVnrArea'] = all_data['MasVnrArea'].fillna(0)

- MSZoning: the general zoning classification
- RL이 최빈값으로 빈 부분은 RL로 채움

In [ ]:
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])

In [ ]:
all_data.info()

In [ ]:
rate = (all_data.isnull().sum() / len(all_data)) * 100
rate = rate.drop(rate[rate == 0].index).sort_values(ascending=False)

missing_data = pd.DataFrame({'Missing Rate':rate})
missing_data

- Utilities: Type of utilities available 

In [ ]:
all_data = all_data.drop(['Utilities'], axis=1)

- Functional

In [ ]:
all_data['Functional'] = all_data['Functional'].fillna('Typ')

- Exterior1st, Exterior2nd, Electrical, KitchenQual, SaleType 모두 최빈값 대체

In [ ]:
for col in ['Exterior1st', 'Exterior2nd', 'Electrical', 'KitchenQual', 'SaleType']:
    all_data[col] = all_data[col].fillna(all_data[col].mode()[0])

In [ ]:
rate = (all_data.isnull().sum() / len(all_data)) * 100
rate = rate.drop(rate[rate == 0].index).sort_values(ascending=False)

missing_data = pd.DataFrame({'Missing Rate':rate})
missing_data

### 형변환

In [ ]:
all_data.info()

In [ ]:
# 수치형 값들 중 범주형의 특성을 가진것 변환

for col in ['MSSubClass', 'OverallQual']:
    all_data[col] = all_data[col].astype(str)